In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
import seaborn as sns
%matplotlib inline
import warnings
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', -1)

In [ ]:
data = pd.read_csv('../input/electric-production/Electric_Production.csv', index_col='DATE', parse_dates=True)
data.index.freq = 'MS'
data.head()


In [ ]:
type(data)

In [ ]:
data.size

In [ ]:
data.describe()

In [ ]:
data_ma=data.rolling(window=12).mean()

In [ ]:
data.plot(figsize=(20, 4))

In [ ]:
fig = plt.subplots(figsize=(20, 4))
ax = sns.boxplot(x=data['Value'],whis=1.5)

In [ ]:
plt.figure(figsize=(20,4))
sns.distplot(data['Value'])
plt.show()

In [ ]:
fig = data.Value.hist(figsize = (20,4))

In [ ]:
data_ma=data.rolling(window=12).mean()

In [ ]:
data_ma.plot(figsize=(20, 4))

In [ ]:
data.head()

In [ ]:
ndata=data.shift(1)
ndata.head()


In [ ]:
data_base=pd.concat([data,ndata],axis=1)
data_base [:10]

In [ ]:
data_base.columns=['Actual_Value','Forcaste_Value']

In [ ]:
data_base.head()

In [ ]:
data_base.iloc[:,:2]

In [ ]:
data_base.dropna(inplace=True)
data_base.head()

In [ ]:
plot_acf(data)

In [ ]:
plot_pacf(data)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

We have 397 row,we will split them into train and test ,by ratio 75% fot train set and 25% for test set

In [ ]:
data_train=data [1:298]
data_test = data[298:397]


Choose initial parameters(p,d,q) randomly

In [ ]:
data_model=ARIMA(data_train,order=(2,0,3))

In [ ]:
data_model_fit = data_model.fit()

In [ ]:
data_model_fit.aic

In [ ]:
data_forecaste=data_model_fit.forecast(steps=99)[0]
data_forecaste [:10]

In [ ]:
np.sqrt(mean_squared_error(data_test,data_forecaste))


In [ ]:
MAEValue = mean_absolute_error(data_base.Actual_Value ,data_base.Forcaste_Value)
MAEValue

In [ ]:
p_values=range(0,5)
d_values=range(0,3)
q_values=range(0,5)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

To choose the best parameters (p,d,q),we use for loop to find the best one

In [ ]:
for p in p_values:
    for d in d_values:
        for q in q_values:
            order=(p,d,q)
            train,test=data [1:298],data [298:397]
            predictions=list()
            for i in range(len(test)):
                try:
                    model=ARIMA(train,order)
                    model_fit=model.fit(disp=0)
                    y_pred=model_fit.forecast()[0]
                    predictions.append(y_pred)
                    error=mean_squared_error(test,predictions)
                    print("ARIMA%s RMSE =%.2f"%(order,error))
                except:   
                    continue
                    